In [5]:
# %load QuantumProgramRunner.py
#!/usr/bin/env python3

import sys
import QuantumCircuits
import ExperimentUtils
import SignalUtils
import random

def run(args):
    experiment = ExperimentUtils.setup_experiment(args)
    program = experiment.qconf.program
    timeout = experiment.qconf.timeout
    if program == "find_period":
        N = int(input("Enter a value for N:\nN = "))
        a = input("Enter a value for a (or type 'rand' for random value between 2..N-1):\na = ")
        if a == "rand":
            a = random.randint(2, N-1)
        else:
            a = int(a)
        def run_expr():
            r = experiment.find_period(a, N)
            print(f"Found period r={r} for a={a} and N={N}.")
        SignalUtils.tryExecuteWithTimeout(run_expr, timeout, f"Failed to find period within timeout: {timeout} seconds.")
        return
    elif program == "factorize_N":
        N = int(input("Enter a value N to factorize:\nN = "))
        def run_expr():
            factors = experiment.factorize_N(N)
            print(f"Found factors: {factors[0]} X {factors[1]} = {N}")
        SignalUtils.tryExecuteWithTimeout(run_expr, timeout, f"Failed to factorize {N} within timeout: {timeout} seconds.")
        return
    else: 
        print(f"FATAL: Failed to find program '{program}'")    

if __name__ == "__main__":
    run(None)

Available backends:
  1: local_qasm_simulator
  2: ibmqx_hpc_qasm_simulator
  3: ibmqx_qasm_simulator
  4: local_qasm_simulator
  5: local_clifford_simulator
  6: local_qiskit_simulator
  7: local_unitary_simulator
  8: ibmqx4
  9: ibmqx5
  10: ibmqx2
Run on which backend?
> 1
Available programs:
  find_period: Takes two integers, a and N, and finds the period of the modular exponentiation function.
  factorize_N: Takes an integer N and finds factors of N using Shor's algorithm.
Run which program?
> factorize_N
Enter a value N to factorize:
N = 15
Selected random value a=6 to find period.
Found common period between N=15 and a=6
Found factors: 3 X 5 = 15
